In [5]:
# Import Statements
import numpy as np
import pandas as pd
import heapq as pq
import csv
import statistics as stats
import copy
import math

# Setting to display all columns of dataframe
pd.set_option('display.max_columns', 30)

# Constants
QUEUE_BOUND = 2
SEARCH_MAX_ITER = 200

# Load countries
def load_countries():
    countries = {}

    with open("countriestest.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: float(value) for name, value in row.items() if name != "Country"}

    return countries

# Load data methods
def load_up():
    
    #load and create df
    countryList = load_countries()
    root_country_df = pd.DataFrame(countryList).transpose()
    
    return root_country_df

def get_country_names(df):
    return df.index.to_list()

# Load dataframes
country_df = load_up()
root_country_df = load_up()
COUNTRY_NAMES = get_country_names(country_df)

# Maslow
L_ONE_RESOURCES = {'one: water': 1, 'one: food': 1,'one: house': 0.25}
L_TWO_RESOURCES = {'two: transit station': 0.05, 'two: municipal center': 0.05,'two: school': 0.15, 'two: university': 0.025, 'two: hospital': 0.10} 
L_THREE_RESOURCES = {'three: stadium': 0.01, 'three: religious center': 0.10, 'three: theatre': 0.25, 'three: park': 0.40, 'three: pet': 0.20, 'three: zoo': 0.25} 
L_FOUR_RESOURCES = {'four: artist': 0.1,'four: politician': 0.1,'four: athlete': 0.1,'four: priest': 0.1,'four: scientist': 0.2, 'four: doctor': 0.1, 'four: curator': 0.1, 'four: park ranger': 0.1}
L_FIVE_RESOURCES = {'five: picasso': 0.05, 'five: constitution': 0.025, 'five: olympic gold': 0.05, 'five: ceremony': 0.10, 'five: nobel prize': 0.01, 'five: vaccine': 0.05, 'five: broadway': 0.25, 'five: monument': 0.15, 'five: animal reserve': 0.10}

# List of levels
LEV_LIST = [L_ONE_RESOURCES, L_TWO_RESOURCES, L_THREE_RESOURCES, L_FOUR_RESOURCES, L_FIVE_RESOURCES]

# Level Function
def leveldf(df, country, level, level_function_accessor):
    
    levelSat = False
    marUtility = False
    wasteRate = False
    mult = []
    average = 0

    for key, value in level_function_accessor[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = True
            
        #marginal utility
        if countryVal > value*3:
            marUtility = True
            
            if countryVal > value * 6:
                wasteRate = True
                
    
    average = stats.mean(mult)
    
    if levelSat:
        average = average*0.01
        
    if marUtility:
        average = average*0.01
        
    if wasteRate:
        average = average*0.00001
        
    return average

# Population Modeling
def pop(df):
    for row in range(len(df)):
        values = df.iloc[row]
        popVal = values[0]
        BR = round(np.random.normal(loc=values[len(values) - 2], scale=1.0, size=None))
        DR = round(np.random.normal(loc=values[len(values) - 1], scale=1.0, size=None))
        df.iat[row, 0] = popVal + BR - DR
        if df.iat[row, 0] <= 0:
            df.iat[row, 0] = 1
            
# Maslow Function
def maslowLevelVals(df, country, level, level_function_accessor):
    
    maslowList = []
    
    ## change population
    pop(df)
    
    ## normalize
    norm_df = df.copy()
    
    for row in range(len(norm_df)):
        
        values = norm_df.iloc[row]
        popVal = values[0]
        
        for vals in range(1, len(values)):
            values[vals] = values[vals]/popVal
    
    # maslow function
    for num in range(1, level+1):
        levValue = leveldf(norm_df, country, num, level_function_accessor)
        maslowList.append(levValue)
            
    return maslowList

def maslowList(lst):
    sumVal = 0
    sumVal = sumVal + lst[0] * 0.10
    sumVal = sumVal + lst[1] * 10
    sumVal = sumVal + lst[2] * 100
    sumVal = sumVal + lst[3] * 1000
    return sumVal

def maslowVal(df, country, level, LEV_LIST):
    
    lst = maslowLevelVals(df, country, level, LEV_LIST)
    val = maslowList(lst)
    return val


# Transforms
FARM_TM = ['FARM', {'population': 1, 'land' : 25, 'water': 25}, {'food': 5, 'population': 1}]

#level one transforms
PROCREATE_TM = ['PRO CREATE', {'base: population': 2, 'population: adult': 2}, {'base: population': 4, 'population: adult': 2, 'population: children': 2}]
GATHERWATER_TM = ['GATHER WATER', {'population: adult': 5, 'base: water': 50, 'water: fresh': 50},  {'population: adult': 5, 'one: water': 50}]
PURIFYWATER_TM = ['PURIFY WATER', {'population: adult': 10, 'base: water': 50, 'water: salt': 50},  {'population: adult': 10, 'one: water': 50}]
FISH_TM = ['PURIFY WATER', {'population: adult': 10, 'base: water': 1, 'water: salt': 1},  {'population: adult': 10, 'base: water': 1, 'water: salt': 1, 'one: food': 25}]
FARM_TM = ['FARM', {'population: adult': 20, 'land: plain': 100, 'one: water': 25},  {'population: adult': 20, 'land: plain': 95, 'land: waste': 5, 'one: food': 200}]
LOGGING_TM = ['LOG TREES', {'population: adult': 15, 'land: forest': 100, 'int: metal': 30},  {'population: adult': 15, 'land: forest': 90,'land: waste': 10, 'int: metal': 20, 'int: wood': 400}]
BUILDHOUSES_TM = ['BUILD HOUSE', {'population: adult': 10, 'int: wood': 200, 'land: zoned': 100},  {'population: adult': 10, 'land: zoned': 100, 'one: house': 300}]
HUNTGATHER_TM = ['HUNT AND GATHER', {'population: adult': 10, 'one: water': 15, 'int: metal': 30},  {'population: adult': 10, 'one: water': 5, 'int: metal': 25, 'one: food': 150}]
MINE_TM = ['MINE', {'population: adult': 30, 'land: quarry': 100, 'one: water': 20, 'one: food': 20},  {'population: adult': 30,'land: quarry': 95,'land: waste': 5, 'one: water': 5, 'int: metal': 25, 'one: food': 150}]

CLEARPLAIN_TM = ['CLEAR PLAIN', {'population: adult': 10,'land: plain': 200, 'one: water': 25, 'one: food': 20}, {'population: adult': 10,'land: zoned': 200}]
CLEARFOREST_TM = ['CLEAR FOREST', {'population: adult': 10,'land: forest': 200, 'one: water': 25, 'one: food': 20}, {'population: adult': 10,'land: zoned': 200}]
CLEARDESERT_TM = ['CLEAR DESERT', {'population: adult': 10,'land: desert': 200, 'one: water': 25, 'one: food': 20}, {'population: adult': 10,'land: zoned': 200}]
CLEARQUARRY_TM = ['CLEAR QUARRY', {'population: adult': 10,'land: quarry': 200, 'one: water': 25, 'one: food': 20}, {'population: adult': 10,'land: zoned': 200}]
CLEARMARSH_TM = ['CLEAR MARSH', {'population: adult': 10,'land: marsh': 200, 'one: water': 25, 'one: food': 20}, {'population: adult': 10,'land: zoned': 200}]
CLEARMOUNTAIN_TM = ['CLEAR MOUNTAIN', {'population: adult': 10,'land: mountain': 200, 'one: water': 25, 'one: food': 20}, {'population: adult': 10,'land: zoned': 200}]

#level two transforms
STATION_TM = ['BUILD TRANSIT STATION', {'population: adult': 100, 'one: water': 25, 'one: food': 50, 'base: energy': 50, 'energy: old': 50,'land: zoned': 50}, {'population: adult': 100, 'two: transit station': 3,'land: zoned': 50}]
MUNICIPALCENTER_TM = ['BUILD MUNICIPAL CENTER', {'population: adult': 100, 'one: water': 50, 'one: food': 100, 'base: energy': 200,'base: energy': 200, 'land: zoned': 25, 'one: house': 25}, {'population: adult': 100, 'two: municipal center': 1, 'land: zoned': 25}]
SCHOOL_TM = ['BUILD SCHOOL', {'population: adult': 50, 'one: water': 50, 'one: food': 10, 'base: energy': 500,'base: energy': 500, 'land: zoned': 50, 'one: house': 10}, {'population: adult': 100, 'two: school': 1, 'land: zoned': 50}]
UNIVERSITY_TM = ['BUILD SCHOOL', {'population: adult': 200, 'one: water': 100, 'one: food': 200, 'base: energy': 1000,'energy: old': 1000, 'land: zoned': 250, 'two: school': 10}, {'population: adult': 200, 'two: university': 1, 'land: zoned': 250}]
HOSPITAL_TM = ['BUILD HOSPITAL', {'population: adult': 70, 'one: water': 50, 'one: food': 100, 'base: energy': 100,'energy: old': 100, 'land: zoned': 70}, {'population: adult': 70, 'two: hospital': 1, 'land: zoned': 70}]

DIGCOAL_TM = ['DIG COAL', {'population: adult': 20, 'land: marsh': 100, 'one: water': 70, 'one: food': 50, 'int: metal': 40}, {'population: adult': 20, 'base: energy': 2500, 'energy: old': 2500, 'int: metal': 40, 'land: marsh': 95,'land: waste': 5, 'five: waste': -10}]
DIGOIL_TM = ['DIG OIL', {'population: adult': 20, 'land: desert': 100, 'one: water': 70, 'one: food': 50, 'int: metal': 40}, {'population: adult': 20, 'base: energy': 5000, 'energy: old': 5000, 'int: metal': 40, 'land: desert': 95,'land: waste': 5, 'five: waste': -10}]


#level three transforms
STADIUM_TM = ['BUILD STADIUM', {'population: adult': 500,'two: municipal center': 3, 'two: transit station': 5, 'one: water': 250, 'one: food': 500, 'base: energy': 100, 'energy: old': 100,'land: zoned': 250}, {'population: adult': 500, 'three: stadium': 1,'land: zoned': 250}]
RELIGIOUSCENTER_TM = ['BUILD RELIGIOUS CENTER', {'population: adult': 200,'two: municipal center': 3, 'two: transit station': 5, 'one: water': 25, 'one: food': 50, 'base: energy': 100, 'energy: old': 100,'land: zoned': 250}, {'population: adult': 500, 'three: religious center': 1,'land: zoned': 250}]
THEATRE_TM = ['BUILD THEATRE', {'population: adult': 100,'one: water': 250, 'one: food': 500, 'base: energy': 50,'energy: old': 50,'land: zoned': 500}, {'population: adult': 500, 'three: theatre': 1,'land: zoned': 500}]
FINDANIMAL_TM = ['FIND ANIMAL', {'population: adult': 100,'one: water': 250, 'one: food': 500, 'base: energy': 50,'energy: old': 50,'land: zoned': 500, 'land: forest': 100}, {'population: adult': 100, 'three: pet': 125,'land: zoned': 500, 'land: forest': 100}]
PARK_TM = ['BUILD PARK', {'population: adult': 50,'one: water': 20, 'one: food': 50, 'base: energy': 20,'energy: old': 20,'land: zoned': 200}, {'population: adult': 500, 'three: park': 10,'land: zoned': 200}]
ZOO_TM = ['BUILD ZOO', {'population: adult': 200,'one: water': 250, 'one: food': 500, 'base: energy': 70,'energy: old': 70,'land: zoned': 500, 'land: mountain': 100}, {'population: adult': 100, 'three: zoo': 2, 'land: mountain': 100}]

#level four transforms
JOB_TM = ['CREATE JOB', {'base: population': 40,'population: children': 40, 'two: university': 2}, {'base: population': 40,'population: adult': 40, 'two: university': 2, 'base: job': 40}]
ARTIST_TM = ['ARTIST', {'base: job': 30, 'two: transit station': 5}, {'base: job': 30, 'four: artist': 30, 'two: transit station': 5}]
POLITICIAN_TM = ['POLITICIAN', {'base: job': 10, 'two: municipal center': 5}, {'base: job': 30, 'four: politician': 10, 'two: municipal center': 5}]
ATHLETE_TM = ['ATHLETE', {'base: job': 30, 'three: stadium': 1}, {'base: job': 30, 'four: athlete': 30, 'three: stadium': 1}]
PRIEST_TM = ['PRIEST', {'base: job': 30, 'three: religious center': 1}, {'base: job': 30, 'four: priest': 30, 'two: religious center': 1}]
ATHLETE_TM = ['ATHLETE', {'base: job': 30, 'three: stadium': 1}, {'base: job': 30, 'four: athlete': 30, 'three: stadium': 1}]
SCIENTIST_TM = ['SCIENTIST', {'base: job': 30, 'two: university': 2}, {'base: job': 30, 'four: scientist': 30, 'two: university': 2}]
DOCTOR_TM = ['DOCTOR', {'base: job': 10, 'two: university': 5}, {'base: job': 10, 'four: scientist': 10, 'two: university': 5}]
DIRECTOR_TM = ['DIRECTOR', {'base: job': 10, 'three: theatre': 2}, {'base: job': 10, 'four: director': 10, 'three: theatre': 2}]
CURATOR_TM = ['CURATOR', {'base: job': 10, 'three: theatre': 2, 'two: university': 1}, {'base: job': 10, 'four: curator': 10, 'three: threatre': 2, 'two: university': 1}]
PARKRANGER_TM = ['PARK RANGER', {'base: job': 10, 'three: pet': 10, 'three: zoo': 2, 'three: park': 15}, {'four: park ranger': 10, 'three: pet': 10, 'three: zoo': 2}]


#level five transforms
CLEANENERGY_TM = ['CLEAN ENERGY', {'four: scientist': 10, 'four: politician': 10,'four: park ranger': 5}, {'base: energy': 10000, 'energy: clean': 10000,'four: scientist': 10, 'four: politician': 10,'job: park ranger': 5}]
PICASSO_TM = ['CREATE PICASSO', {'four: artist': 50, 'base: energy': 1000, 'energy: clean': 1000}, {'five: picasso': 1, 'four: artist': 50}]
CONSTITUTION_TM = ['CREATE CONSTITUTION', {'four: politician': 50, 'base: energy': 1000, 'energy: clean': 1000}, {'five: constitution': 1, 'four: politician': 50}]
OLYMPICGOLD_TM = ['CREATE OLYMPIC GOLD', {'four: athlete': 50, 'base: energy': 1000, 'energy: clean': 1000}, {'five: olympic gold': 1, 'four: athlete': 50}]
CEREMONY_TM = ['CONDUCT CEREMONY', {'four: priest': 10, 'base: energy': 100, 'energy: clean': 100}, {'five: ceremony': 1, 'four: priest': 10}]
NOBELPRIZE_TM = ['WIN NOBEL PRIZES', {'four: scientist': 100, 'base: energy': 500, 'energy: clean': 500}, {'five: nobel prize': 1, 'four: scientist': 100}]
VACCINE_TM = ['MANUFACTURE VACCINE', {'four: doctor': 75, 'base: energy': 500, 'energy: clean': 500}, {'five: vaccine': 1, 'four: doctor': 75}]
BROADWAY_TM = ['DEVELOP BROADWAY', {'four: director': 15, 'base: energy': 500, 'energy: clean': 500}, {'five: broadway': 1, 'four: director': 15}]
MONUMENT_TM = ['BUILD MONUMENT', {'four: curator': 15, 'base: energy': 500, 'energy: clean': 500}, {'five: monument': 1, 'four: curator': 15}]
ANIMALRESERVE_TM = ['BUILD ANIMAL RESERVE', {'four: park ranger': 25, 'base: energy': 700, 'energy: clean': 700}, {'five: animal reserve': 1, 'four: park ranger': 25}]

FIXPLAIN_TM = ['FIX WASTELAND TO PLAIN', {'land: waste': 50,'base: energy': 500, 'energy: clean': 500}, {'land: plain': 50}]
FIXFOREST_TM = ['FIX WASTELAND TO FOREST', {'land: waste': 50,'base: energy': 500, 'energy: clean': 500}, {'land: forest': 50}]
FIXDESERT_TM = ['FIX WASTELAND TO DESERT', {'land: waste': 50,'base: energy': 500, 'energy: clean': 500}, {'land: desert': 50}]
FIXMOUNTAIN_TM = ['FIX WASTELAND TO MOUNTAIN', {'land: waste': 50,'base: energy': 500, 'energy: clean': 500}, {'land: mountain': 50}]
FIXQUARRY_TM = ['FIX WASTELAND TO QUARRY', {'land: waste': 50,'base: energy': 500, 'energy: clean': 500}, {'land: quarry': 50}]
FIXMARSH_TM = ['FIX WASTELAND TO MARSH', {'land: waste': 50,'base: energy': 500, 'energy: clean': 500}, {'land: marsh': 50}]

WASTECONVERT_TM = ['FIX WASTELAND', {'five: waste': 500}, {'int: metal': 500, 'int: wood': 500}]

ALL_TEMPLATES_TRANSFORM = [PROCREATE_TM, GATHERWATER_TM, PURIFYWATER_TM, FISH_TM, FARM_TM, LOGGING_TM, BUILDHOUSES_TM, HUNTGATHER_TM, MINE_TM, CLEARFOREST_TM, CLEARFOREST_TM,
                          CLEARDESERT_TM, CLEARQUARRY_TM, CLEARMARSH_TM, CLEARMOUNTAIN_TM, STATION_TM, MUNICIPALCENTER_TM, SCHOOL_TM, UNIVERSITY_TM,
                          HOSPITAL_TM, DIGCOAL_TM, DIGOIL_TM, STADIUM_TM, RELIGIOUSCENTER_TM, THEATRE_TM, PARK_TM, ZOO_TM, FINDANIMAL_TM,
                          JOB_TM, ARTIST_TM, POLITICIAN_TM, ATHLETE_TM, PRIEST_TM, SCIENTIST_TM, DOCTOR_TM, DIRECTOR_TM, CURATOR_TM, PARKRANGER_TM,
                          CLEANENERGY_TM, PICASSO_TM, CONSTITUTION_TM, OLYMPICGOLD_TM, CEREMONY_TM, NOBELPRIZE_TM, VACCINE_TM, BROADWAY_TM, MONUMENT_TM, ANIMALRESERVE_TM,
                          FIXPLAIN_TM, FIXFOREST_TM, FIXDESERT_TM, FIXMOUNTAIN_TM, FIXQUARRY_TM, FIXMARSH_TM, WASTECONVERT_TM]


def transform(df, country, transform_template):
    
    allowed = True
    
    #check if transform is possible
    for key in transform_template[1]:
        val = transform_template[1][key]
        if(df.loc[country, key] - val < 0):
            allowed = False
    
    if(allowed):
        #remove input resoures
        for key in transform_template[1]:
            val = transform_template[1][key]
            df.loc[country, key] -= val

        #add output resources
        for key in transform_template[2]:
            val = transform_template[2][key]
            df.loc[country, key] += val
    return allowed


# Transfers
FOOD_TR = ['FOOD', {'one: food': 5}]
WATER_TR = ['WATER', {'one: water': 5}]
TIMBER_TR = ['WOOD', {'int: wood': 5}]
METAL_TR = ['METAL', {'int: metal': 5}]

ALL_TEMPLATES_TRANSFER = [FOOD_TR,WATER_TR,TIMBER_TR,METAL_TR]

def transfer(df, country1, country2, transfer_template):

    allowed = True

    #check for validity
    
    if country1==country2:
        allowed = False

    for key in transfer_template[1]:
        val = transfer_template[1][key]
        if (df.loc[country1,key] - val < 0):
            allowed = False
            
    if(allowed):
        #remove resource from country 1
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country1, key] -= val

        #add resource to country 2
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country2, key] += val
            
    return allowed

# Trade
def trade(df, country1, country2, transfer1, transfer2):
    # A trade deal between two countries country1 and country2
    # for which country uses transfer1 and country2 uses transfer2
    if transfer1 == transfer2:
        return False
    if transfer(df, country1, country2, transfer1):
        if not transfer(df, country2, country1, transfer2):
            transfer(df, country2, country1, transfer1)
            return False
        else:
            return True
    else:
        return False
    

# Node & Priority Queue Def
class Node:
    def __init__(self, state_quality, exp_util, sched, countrydf):
        self.state_quality = state_quality
        self.exp_util = exp_util
        self.sched = sched
        self.countrydf = countrydf
    def getSqual(self):
        return self.state_quality
    def getExutil(self):
        return self.exp_util
    def getSched(self):
        return self.sched
    def getCountrydf(self):
        return self.countrydf
    def __lt__(self, other):
        return self.exp_util < other.exp_util
    def __str__(self):
        s = "Node with State Quality: {}\nExpected Utility: {}\nSchedule: {}\nDataframe:\n{}".format(self.state_quality, self.exp_util, self.sched, self.countrydf)
        return s
    
class BoundedPriorityQueue:
    # can change bound
    def __init__(self, queue=list(), bound=QUEUE_BOUND):
        self.bound = bound
        self.queue = queue
    def push(self, priority, item):
        if len(self.queue) == self.bound:
            pq.heappushpop(self.queue, (priority, item))
        else:
            pq.heappush(self.queue, (priority, item))
    def pop(self):
        if len(self.queue) >= 1:
            return pq.heappop(self.queue)[-1]
        print("no nodes on fringe")
    ##maybe delete    
    def items(self):
        return list(item for _,_, item in self.queue)
    def clear(self):
        self.queue.clear()
    def len(self):
        return len(self.queue)
    def getState(self, index):
        if len(self.queue) > index:
            return self.queue[index][1]
        print("index out of bound")
    def pState(self, index):
        if len(self.queue) > index:
            print(self.queue[index][1])
        print("index out of bound")
    def getPriority(self, index):
        if len(self.queue) > index:
            return self.queue[index][0]
        print("index out of bound")
    def popBest(self):
        if len(self.queue) >= 1:
            return pq.nlargest(1, self.queue)[0][1]
        print("no nodes on fringe")
    def copy(self):
        return BoundedPriorityQueue(self.queue.copy())


# Generate Successors
def generate_successors(parentNode, my_country, LEV_LIST):
    
    successors = BoundedPriorityQueue()
    prevSched = parentNode.getSched()
    
    for resource in ALL_TEMPLATES_TRANSFORM:
        
        new_df = parentNode.getCountrydf().copy()
        
        if transform(new_df, my_country, resource):
            state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
            new_schedule = prevSched.copy()
            new_state = Node(state_qual, 0, new_schedule, new_df)
            new_state.sched.append("TRANSFORM: " + resource[0])
            new_state.exp_util = expected_utility_transform(new_state, my_country, LEV_LIST)
            successors.push(new_state.exp_util, new_state)


    for resource_1 in ALL_TEMPLATES_TRANSFER:
        for resource_2 in ALL_TEMPLATES_TRANSFER:
            for other_country in COUNTRY_NAMES:
                
                new_df = parentNode.getCountrydf().copy()
                
                if trade(new_df, my_country, other_country, resource_1, resource_2):
                    state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
                    state_qual_other = maslowVal(new_df, other_country, 5, LEV_LIST)
                    new_schedule = prevSched.copy()
                    new_state = Node(state_qual, 0, new_schedule, new_df)
                    trade_name = "TRADE: " + my_country + " GIVES " + resource_1[0] + ", " + other_country + " GIVES " + resource_2[0]
                    new_state.sched.append(trade_name)
                    new_state.exp_util = expected_utility_trade(L, k, x_0, new_state, my_country, LEV_LIST, other_country)
                    successors.push(new_state.exp_util, new_state)
                    
    return successors

# Reward Functions
def undiscounted_reward(node, my_country,LEV_LIST):
    root_node_score = maslowVal(root_country_df, my_country, 5, LEV_LIST)
    cur_node_score = maslowVal(node.getCountrydf(), my_country, 5, LEV_LIST)
    return cur_node_score - root_node_score

GAMMA = 0.05

def discounted_reward(node, my_country, LEV_LIST):
    return (GAMMA ** len(node.getSched()))*undiscounted_reward(node, my_country, LEV_LIST)

L = 1.0
k = 1.0
x_0 = 0.0

def probability(L, k, node, other_country, LEV_LIST, x_0):
    return (L/(1+math.exp(-k*(discounted_reward(node, other_country, LEV_LIST)-x_0))))

COST_FAILURE = -0.05

def expected_utility_trade(L, k, x_0, node, my_country, LEV_LIST, other_country):
    prob = probability(L, k, node, other_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, other_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    other_country_utility = success + fail
    prob = probability(L, k, node, my_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, my_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    my_country_utility = success + fail
    total = (other_country_utility+my_country_utility)/2
    return total

PROB = 0.95
COST_FAILURE = -0.05
def expected_utility_transform(node, my_country, LEV_LIST):
    success = PROB * discounted_reward(node, my_country, LEV_LIST)
    fail = (1-PROB)*COST_FAILURE 
    return success + fail

# Search Function
def search(root_node, my_country,level_accessor):
    
    num = 1
    
    fringe = generate_successors(root_node, my_country, level_accessor)
    newfringe = BoundedPriorityQueue()
    for i in range(1, SEARCH_MAX_ITER):
        fringe_bound = fringe.len()
        for j in range(0,fringe_bound):
            successors = generate_successors(fringe.pop(), my_country, level_accessor)
            for k in range(0,successors.len()-1):
                newfringe.push(successors.getState(k).getExutil(),successors.getState(k))
                print("Explored Node: " + str(num))
                num = num + 1
        print("Level: " + str(i))
        print(newfringe.popBest())
        fringe = newfringe.copy()
        newfringe.clear()
    return fringe.popBest()


# Executable Code

In [6]:
state_qual = 1.2
exp_util = 3.0
sched = ['ROOT STATE']
test_node = Node(state_qual, exp_util, sched, country_df)
nodeBest = search(test_node, 'The Vale', LEV_LIST)
print(nodeBest)

Explored Node: 1
Explored Node: 2
Level: 1
Node with State Quality: 0.00016666666666666666
Expected Utility: -0.0024995960884353764
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER']
Dataframe:
                base: population  population: adult  population: children  \
Atlantis                    18.0               20.0                   0.0   
Dinotopoia                  53.0               45.0                   5.0   
Erewhon                     29.0               20.0                  10.0   
King's Landing             197.0              150.0                  50.0   
The Vale                    98.0               90.0                  10.0   

                base: land  land: plain  land: forest  land: desert  \
Atlantis           10000.0       2000.0        2000.0        2000.0   
Dinotopoia         20000.0       9000.0        2000.0        1000.0   
Erewhon            20000.0       4000.0        2000.0        4000.0   
King's Landing      5000.0        500.0        1500.0     

Explored Node: 7
Explored Node: 8
Level: 4
Node with State Quality: 0.0011054421768707484
Expected Utility: -0.002499999671821856
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: FARM', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER']
Dataframe:
                base: population  population: adult  population: children  \
Atlantis                    16.0               20.0                   0.0   
Dinotopoia                  55.0               45.0                   5.0   
Erewhon                     32.0               20.0                  10.0   
King's Landing             200.0              150.0                  50.0   
The Vale                    98.0               90.0                  10.0   

                base: land  land: plain  land: forest  land: desert  \
Atlantis           10000.0       2000.0        2000.0        2000.0   
Dinotopoia         20000.0       9000.0        2000.0        1000.0   
Erewhon            20000.0       4000.0        2000.0     

Explored Node: 12
Level: 6
Node with State Quality: 0.0014166666666666668
Expected Utility: -0.0024999999989047603
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER']
Dataframe:
                base: population  population: adult  population: children  \
Atlantis                    18.0               20.0                   0.0   
Dinotopoia                  57.0               45.0                   5.0   
Erewhon                     27.0               20.0                  10.0   
King's Landing             196.0              150.0                  50.0   
The Vale                    96.0               90.0                  10.0   

                base: land  land: plain  land: forest  land: desert  \
Atlantis           10000.0       2000.0        2000.0        2000.0   
Dinotopoia         20000.0       9000.0        2000.0        1000.0   
Erewhon            

Explored Node: 17
Explored Node: 18
Level: 9
Node with State Quality: 0.0018617021276595746
Expected Utility: -0.0024999999999998296
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER']
Dataframe:
                base: population  population: adult  population: children  \
Atlantis                    19.0               20.0                   0.0   
Dinotopoia                  57.0               45.0                   5.0   
Erewhon                     30.0               20.0                  10.0   
King's Landing             192.0              150.0                  50.0   
The Vale                    94.0               90.0                  10.0   

                base: land  land: plain  land: forest  land: desert  \
Atlantis           10000.0       2000.0        2000.0        2

Explored Node: 22
Level: 11
Node with State Quality: 0.0017799352750809067
Expected Utility: -0.002500000000000002
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PRO CREATE']
Dataframe:
                base: population  population: adult  population: children  \
Atlantis                    10.0               20.0                   0.0   
Dinotopoia                  63.0               45.0                   5.0   
Erewhon                     21.0               20.0                  10.0   
King's Landing             191.0              150.0                  50.0   
The Vale                   103.0               90.0                  12.0   

                base: land  land: plain  land: forest  land: desert  \
Atlantis           10000.0  

...

Explored Node: 388
Level: 194
Node with State Quality: 0.0004428341384863124
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREA

Explored Node: 390
Level: 195
Node with State Quality: 0.00044176706827309245
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CRE

Explored Node: 392
Level: 196
Node with State Quality: 0.00043859649122807024
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CRE

Explored Node: 395
Explored Node: 396
Level: 198
Node with State Quality: 0.00043238993710691825
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 

Explored Node: 397
Explored Node: 398
Level: 199
Node with State Quality: 0.0004303599374021909
Expected Utility: -0.0025000000000000022
Schedule: ['ROOT STATE', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: FARM', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: GATHER WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PURIFY WATER', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', 'TRANSFORM: PRO CREATE', '